In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
sess = tf.Session()

K.set_session(sess)

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [6]:
all_elems_equal = tf.reduce_all(tf.equal(tf.shape(t)[1], tf.shape(t)[1]))

NameError: name 't' is not defined

In [6]:
y_true_1 = gt1_proj[:,:,:18]
y_pred_1 = pred_1_proj[:,:,18:]
y_true_2 = gt1_proj[:,:,18:]
y_pred_2 = pred_1_proj[:,:,18:]

In [7]:
batch_size = tf.shape(y_pred_1)[0]

In [8]:
y_true_1d = np.concatenate([y_true_1,y_true_1])
y_pred_1d = np.concatenate([y_pred_1,y_pred_1])
y_true_2d = np.concatenate([y_true_2,y_true_2])
y_pred_2d = np.concatenate([y_pred_2,y_pred_2])

In [109]:
batch_size = 1
def finder(tensor, matches):
    return tensor[:,matches,:]

def matcher(y_true_1,y_pred_1,y_pred_2, bsz):
    result = 0
    for i in range(bsz+1):
        print(result)
        res = np.empty(y_pred_2.shape)
        filterer = np.where(y_true_1d[i,:,-1]!=99)
        y_true_1d_new = y_true_1d[i,filterer,:]
        iou_out = tf.py_func(iou, [y_true_1d_new[0,:,-16:-12],tf.convert_to_tensor(y_pred_1d[i,:,-16:-12])], tf.float64)
        bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)
        out = tf.gather(y_pred_2d[i,:,:], [bipartite_matches], axis=0)
        if i != 0:
            K.concatenate([result,out], axis=-1)
            print("out: ",i)
        else:
            print("entry: ",i)
            result = out
            print("entry: ",K.eval(result).shape)
    return result

In [110]:
result = matcher(y_true_1d,y_pred_1d,y_pred_2d,1)
K.eval(result).shape

0
entry:  0
entry:  (1, 4, 18)
Tensor("Gather_53:0", dtype=float32)
out:  1


(1, 4, 18)